In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Loading Raw Data...")
df_raw = pd.read_csv('../data/raw/flights.csv')

df = df_raw.copy()

print(f"Loaded {len(df):,} rows * {len(df.columns)}columns")
print(f"Raw data preserved in df_raw")

Loading Raw Data...
Loaded 5,819,079 rows * 31columns
Raw data preserved in df_raw


In [2]:
print("MISSING VALUES ANALYSIS:")
print("="*80)

missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing': missing.values,
    'Percent': missing_percent.values
})
missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Percent', ascending=False)

print(f"Columns with missing values: {len(missing_df)}\n")
print(missing_df.to_string(index=False))

print("\n" + "="*80)
print("CLEANING STRATEGY:")
print("="*80)

if 'CANCELLED' in df.columns:
    cancelled_count = df['CANCELLED'].sum()
    print(f"1. Removing {cancelled_count:,} cancelled flights")
    df = df[df['CANCELLED'] == 0].copy()
    print(f"  Remaining flights: {len(df):,} rows")

    delay_cols = ['DEPARTURE DELAY', 'ARRIVAL_DELAY']
    for col in delay_cols:
        if col in df.columns:
            missing_before = df[col].isnull().sum()
            df[col].fillna(0, inplace=True)
            print(f"2. Filled {missing_before:,} missing values in '{col}' with 0")

print("\n Basic cleaning complete")
print(f"Final dataset: {len(df):,} rows * {len(df.columns)} columns")

MISSING VALUES ANALYSIS:
Columns with missing values: 18

             Column  Missing   Percent
CANCELLATION_REASON  5729195 98.455357
      WEATHER_DELAY  4755640 81.724960
LATE_AIRCRAFT_DELAY  4755640 81.724960
      AIRLINE_DELAY  4755640 81.724960
     SECURITY_DELAY  4755640 81.724960
   AIR_SYSTEM_DELAY  4755640 81.724960
       ELAPSED_TIME   105071  1.805629
           AIR_TIME   105071  1.805629
      ARRIVAL_DELAY   105071  1.805629
       ARRIVAL_TIME    92513  1.589822
            TAXI_IN    92513  1.589822
          WHEELS_ON    92513  1.589822
         WHEELS_OFF    89047  1.530259
           TAXI_OUT    89047  1.530259
     DEPARTURE_TIME    86153  1.480526
    DEPARTURE_DELAY    86153  1.480526
        TAIL_NUMBER    14721  0.252978
     SCHEDULED_TIME        6  0.000103

CLEANING STRATEGY:
1. Removing 89,884 cancelled flights
  Remaining flights: 5,729,195 rows
2. Filled 15,187 missing values in 'ARRIVAL_DELAY' with 0

 Basic cleaning complete
Final dataset: 5,729,195

In [3]:
print("CREATING DERIVED FEATURES:")

df['IS DELAYED'] = (df['ARRIVAL_DELAY']> 15).astype(int)
delayed_count = df['IS DELAYED'].sum()
delay_rate = (delayed_count / len(df)) * 100
print(f"1. IS_DELAYED column created")
print(f"  {delayed_count:,} delayed flights ({delay_rate:.2f}%)")

def categorize_delay(delay):
    if delay <= 0:
        return 'On Time'
    elif delay <= 15:
        return 'Slightly Delayed (<15 min)'
    elif delay <= 60:
        return 'Moderately Delayed (15-60 min)'
    else:
        return 'Severely Delayed (0-60 min)' 
    
df['DELAY CATEGORY'] = df['ARRIVAL_DELAY'].apply(categorize_delay)
print("2. DELAY_CATEGORY column created with categories:")
print(df['DELAY CATEGORY'].value_counts())

def categorize_time(scheduled_dep):
    if pd.isna(scheduled_dep):
        return 'Unknown'
    
    hour = int(scheduled_dep // 100)
    if 5 <= hour < 12:
        return 'Morning (5am-12pm)'
    elif 12 <= hour < 17:
        return 'Afternoon (12pm-5pm)'
    elif 17 <= hour < 21:
        return 'Evening (5pm-9pm)'
    else:
        return 'Night (9pm-5am)'
    

if 'SCHEDULED DEPARTURE' in df.columns:
    df['TIME OF DAY'] = df['SCHEDULED DEPARTURE'].apply(categorize_time)
    print(f"\n3. TIME_OF_DAY column created")
    print(df['TIME OF DAY'].value_counts())

print("\n" + "="*80)
print(f" Feature engineering complete")
print(f"New columns added: IS_DELAYED, DELAY_CATEGORY, TIME_OF_DAY")

CREATING DERIVED FEATURES:
1. IS_DELAYED column created
  1,023,498 delayed flights (17.86%)
2. DELAY_CATEGORY column created with categories:
DELAY CATEGORY
On Time                           3642299
Slightly Delayed (<15 min)        1063398
Moderately Delayed (15-60 min)     704406
Severely Delayed (0-60 min)        319092
Name: count, dtype: int64

 Feature engineering complete
New columns added: IS_DELAYED, DELAY_CATEGORY, TIME_OF_DAY


In [4]:
print("SELECTING RELEVANT COLUMNS")

columns_to_keep = [ 'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK',
                   'AIRLINE', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DPEARTURE_TIME', 'DEPARTURE_DELAY', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DISTANCE', 'IS_DELAYED', 'DELAY_CATEGORY', 'TIME_OF_DAY']

columns_to_keep = [col for col in columns_to_keep if col in df.columns]

print(f"Original Columns: {len(df.columns)}")
print(f"Keeping: {len(columns_to_keep)} columns")
print(f"Dropping: {len(df.columns) - len(columns_to_keep)} columns")

df_clean = df[columns_to_keep].copy()

print(f"\n Clean dataset created")
print(f"Shape: {df_clean.shape[0]:,} rows * {df_clean.shape[1]} columns")

print("\nColumns in clean dataset:")
for i, col in enumerate(df_clean.columns, 1):
    print(f"{i}. {col}")

SELECTING RELEVANT COLUMNS
Original Columns: 33
Keeping: 14 columns
Dropping: 19 columns

 Clean dataset created
Shape: 5,729,195 rows * 14 columns

Columns in clean dataset:
1. YEAR
2. MONTH
3. DAY
4. DAY_OF_WEEK
5. AIRLINE
6. FLIGHT_NUMBER
7. ORIGIN_AIRPORT
8. DESTINATION_AIRPORT
9. SCHEDULED_DEPARTURE
10. DEPARTURE_DELAY
11. SCHEDULED_ARRIVAL
12. ARRIVAL_TIME
13. ARRIVAL_DELAY
14. DISTANCE


In [5]:
import os 

print("SAVING CLEANED DATA")
print("="*80)

output_path = '../data/processed/flights_clean.csv'

print(f"Writing to: {output_path}")
print("This may take 30-60 seconds...")

df_clean.to_csv(output_path, index=False)

if os.path.exists(output_path):
    file_size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"\n File saved successfully!")
    print(f" Location: {output_path}")
    print(f" Size: {file_size_mb:.2f} MB")
    print(f" Rows: {len(df_clean):,}")
    print(f" Columns: {len(df_clean.columns)}")
else:
    print("Error: File was not saved successfully.")

print("\n" + "="*80)
print("DATA CLEANING COMPLETE!")
print("="*80)
print("\nCleaning Summary:")
print(f"  - Started with: {len(df_raw):,} rows")
print(f"  - Removed cancelled flights")
print(f"  - Handled missing values")
print(f"  - Created 3 new features")
print(f"  - Final dataset: {len(df_clean):,} rows × {len(df_clean.columns)} columns")
print(f"\n✓ Ready for Phase 3 (Analysis & Visualization)")

SAVING CLEANED DATA
Writing to: ../data/processed/flights_clean.csv
This may take 30-60 seconds...

 File saved successfully!
 Location: ../data/processed/flights_clean.csv
 Size: 320.22 MB
 Rows: 5,729,195
 Columns: 14

DATA CLEANING COMPLETE!

Cleaning Summary:
  - Started with: 5,819,079 rows
  - Removed cancelled flights
  - Handled missing values
  - Created 3 new features
  - Final dataset: 5,729,195 rows × 14 columns

✓ Ready for Phase 3 (Analysis & Visualization)
